In [1]:
# General imports
import os
import numpy as np
import matplotlib.pyplot as plt

# Seg Map imports
from astropy.table import Table
from astropy.io import fits
from tqdm import tqdm

os.chdir("/Users/keith/astr/research_astr/summer-roman-project/4) FOV0_sims/fits")

In [2]:
# Read in Table and sort with highest mag first
hlss_tbl = Table.read("/Users/keith/astr/research_astr/FOV0/catalogs/MOT_SCA1_roll_0_dither_0x_0y_cut_zcut.txt",
                      format='ascii')
hlss_tbl = hlss_tbl.group_by("MODIMAGE")
hlss_tbl.groups[0].sort("MAG_F1500W", reverse=True)
hlss_tbl.groups[1].sort("MAG_F1500W", reverse=True)

# Store len and instantiate empty seg_map
total_obj = len(hlss_tbl)
seg_map = np.zeros((4088,4088))

In [3]:
# Define ellipse functions
A = lambda theta, a, b: ((np.sin(theta)**2)/(a**2)) + ((np.cos(theta)**2)/(b**2))
B = lambda theta, a, b: 2 * np.sin(theta) * np.cos(theta) * ((1/b**2) - (1/a**2))
C = lambda theta, a, b: ((np.sin(theta)**2)/(b**2)) + ((np.cos(theta)**2)/(a**2))

gen_ellipse = lambda x,y,x_0,y_0, ell: (A(*ell) * (x - x_0)**2) + (B(*ell) * (x - x_0) * (y - y_0)) + (C(*ell) * (y - y_0)**2)

In [4]:
# Create meshgrid; meshgrid acts like seg_map pixel indicies
y = np.arange(4088)
x = np.arange(4088)
x_grid, y_grid = np.meshgrid(x,y)
del x
del y

# Loop through all objects, tracking how many are done
ii = 1
for object in tqdm(hlss_tbl):

    # Take object info
    num = object["NUMBER"]
    mag = object["MAG_F1500W"]
    a = object["A_IMAGE"]
    b = object["B_IMAGE"]
    theta = object["THETA_IMAGE"]
    x_0 = object["X_IMAGE"]
    y_0 = object["Y_IMAGE"]
    modimage = object["MODIMAGE"]

    # if STAR
    if modimage:
        circ = lambda x,y: (x-x_0)**2 + (y-y_0)**2
        condition = circ(x_grid, y_grid) <= -233.4 * np.log10(47.44 * mag)
    
    # else GALAXY
    else:
        # Find all values within object's ellipse
        ellipse = lambda x,y: gen_ellipse(x, y, x_0, y_0, (theta, a, b))
        condition = ellipse(x_grid, y_grid) <= 9 # 2 / sqrt(2ln2)

    # # Set all values equal to object id
    seg_map[np.where(condition)] = num
    ii += 1

100%|██████████| 10763/10763 [20:18<00:00,  8.83it/s]


In [5]:
# Show seg map
plt.imshow(seg_map, origin='lower', vmin=0, vmax=1, interpolation='nearest')

In [6]:
raise Exception("Pause")

Exception: Pause

In [ ]:
# Save seg_map
ready_direct = "/Users/keith/astr/research_astr/summer-roman-project/4) FOV0_sims/fits/ready_direct_GRS_FOV0_roll0_dx0_dy0_SCA1_direct_final.fits"
seg_from_MOT = "gaussian_seg.fits"

# Take header info from prepared direct image
file = fits.open(ready_direct)
hdr = file[1].header
file.close()

fits.writeto(seg_from_MOT, data=np.rot90(seg_map, k=3), header=hdr, overwrite=True)